In [ ]:
!pip install shapely

In [ ]:
!pip install schedule

OVAL GEOJSON GENERATOR


In [8]:
import json
import requests
import schedule
import time
import os
from datetime import datetime, timedelta
from shapely.geometry import Polygon
from shapely.geometry.polygon import orient

# Function to convert RGB to Hex
def rgb_to_hex(rgb_str):
    """
    Convert RGB color string in format 'rgb(r, g, b)' to HEX color code.

    Parameters:
    rgb_str (str): RGB color string in format 'rgb(r, g, b)'

    Returns:
    str: HEX color code
    """
    # Extract numeric values from the string
    rgb_values = rgb_str.split('(')[1].split(')')[0].split(',')
    r = int(rgb_values[0].strip())
    g = int(rgb_values[1].strip())
    b = int(rgb_values[2].strip())


    # Clamp RGB values within 0-255 range
    r = max(0, min(r, 255))
    g = max(0, min(g, 255))
    b = max(0, min(b, 255))

    # Convert RGB to HEX
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)



def douglas_peucker(coords, tolerance):
    """Douglas-Peucker algorithm for simplifying coordinates."""
    from math import sqrt

    def perpendicular_distance(pt, start, end):
        # Calculate the perpendicular distance from pt to line defined by start-end
        if start == end:
            return sqrt((pt[0] - start[0]) ** 2 + (pt[1] - start[1]) ** 2)
        else:
            numerator = abs((end[1] - start[1]) * pt[0] - (end[0] - start[0]) * pt[1] + end[0] * start[1] - end[1] * start[0])
            denominator = sqrt((end[1] - start[1]) ** 2 + (end[0] - start[0]) ** 2)
            return numerator / denominator

    def recursive_douglas_peucker(coords, tolerance, start=0, end=None):
        if end is None:
            end = len(coords) - 1

        if end <= start + 1:
            return [coords[start], coords[end]]

        max_distance = 0
        max_index = 0

        for i in range(start + 1, end):
            distance = perpendicular_distance(coords[i], coords[start], coords[end])
            if distance > max_distance:
                max_distance = distance
                max_index = i

        if max_distance >= tolerance:
            first_part = recursive_douglas_peucker(coords, tolerance, start, max_index)
            second_part = recursive_douglas_peucker(coords, tolerance, max_index, end)
            return first_part[:-1] + second_part
        else:
            return [coords[start], coords[end]]

    return recursive_douglas_peucker(coords, tolerance)


# Function to ensure proper order, closure, and orientation of coordinates
def process_coordinates(coords):
    # Ensure proper order and closure
    if coords[0] != coords[-1]:
        coords.append(coords[0])  # Close the polygon if not closed already

    # Orient the polygon
    poly = Polygon(coords)
    oriented_coords = list(orient(poly).exterior.coords)

    oriented_coords.reverse()

    # Convert coordinates to the correct format for GeoJSON
    return [[round(x, 6), round(y, 6)] for x, y in oriented_coords]

# Function to check if the polygon needs to be reversed (for poles)
def is_polygon_reversed(coords):
    # Determine if the polygon should be reversed based on its centroid
    # Example logic: Check if the centroid latitude is above or below a certain threshold
    centroid_y = sum(coord[1] for coord in coords) / len(coords)
    if centroid_y > 0:  # Adjust this threshold as needed
        return True
    else:
        return False


# Function to simplify coordinates using Douglas-Peucker algorithm
def simplify_coordinates(coords, tolerance=0.001):
    simplified_coords = douglas_peucker(coords, tolerance)
    return simplified_coords

# Input and output file paths
input_file = 'C:/Users/alundkvi/Documents/work/scripts/oval-dataTimeTest.json'
output_file = 'C:/Users/alundkvi/Documents/work/scripts/oval_asset_output_not_simplified/output.geojson'

# Function to process each entry and generate GeoJSON-like asset
def process_entry(entry):
    fill_color = entry.get('fill_color', None)
    paths = entry.get('paths', [])

    # Validate input structure
    if not fill_color or not paths:
        return None

    # Convert RGB to Hex
    fill_color_hex = rgb_to_hex(fill_color)

    optimized_paths = []

    # Process each path
    for path in paths:
        optimized_path = []
        
        # Modify each coordinate in the path and optimize
        for coord in path:
            lat = coord.get('lat', None)
            lng = coord.get('lng', None)

            # Skip if coordinates are in the northern hemisphere
            if lat is not None and lat > 0:
                continue

            # Swap lat and lng, subtract 400 from lng
            if lat is not None and lng is not None:
                optimized_path.append([
                    lng - 360,
                    lat
                ])

        # Simplify coordinates if there are valid points in optimized_path
        if optimized_path:
            simplified_coords = simplify_coordinates(optimized_path)
            final_coords = process_coordinates(simplified_coords)
            optimized_paths.append(final_coords)

    # Construct GeoJSON-like asset structure if optimized_paths is not empty
    if optimized_paths:
        asset = {
            "type": "Feature",
            "properties": {
                "stroke": fill_color_hex,
                "stroke-width": 2,
                "stroke-opacity": 1,
                "fill": fill_color_hex,
                "fill-opacity": 0.75
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": optimized_paths
            }
        }

        return asset
    else:
        return None

# Function to fetch data from URL
def fetch_data_and_process(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from {url}")
        return []   

# Function to run the process for each 15-minute interval
#def run_process():
# Define the time range
start_time = datetime(2024, 5, 10, 15, 0)
end_time = datetime(2024, 5, 12, 0, 0)

current_time = start_time
interval_minutes = 15

while current_time <= end_time:
    formatted_time = current_time.strftime("%Y-%m-%d-%H-%M")
    folder_name = formatted_time.replace(':', '-')
    output_folder = f'C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/{folder_name}/'

    # Create output folder if it doesn't exist
    try:
        os.makedirs(output_folder, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {output_folder}: {e}")
        continue
    # Fetch data from URL
    url = f'https://aurorasaurus.org/oval-data?end_date={current_time.strftime("%Y-%m-%dT%H:%M%z")}&local_offset=0'
    data = fetch_data_and_process(url)
    
    display(f"URL: {url}")

    if data:
        # Process each entry and generate GeoJSON-like asset
        for index, entry in enumerate(data):
            asset = process_entry(entry)
            if asset:
                # Generate output file path
                output_file = f'{output_folder}output{index + 1}.geojson'

                # Write GeoJSON-like asset to output file
                try:
                    with open(output_file, 'w') as out_f:
                        json.dump(asset, out_f, indent=2)
                    print(f'Processed entry {index + 1} saved to {output_file}')
                except Exception as e:
                    print(f"Error writing to {output_file}: {e}")
            else:
                print(f'Skipped entry {index + 1} due to missing or invalid data')

    # Move to the next interval
    current_time += timedelta(minutes=interval_minutes)

# Schedule the job every 15 minutes
#schedule.every(1).minutes.do(run_process)

# Run indefinitely
#while True:
#    schedule.run_pending()
#    time.sleep(1)

'URL: https://aurorasaurus.org/oval-data?end_date=2024-05-10T15:00&local_offset=0'

Skipped entry 1 due to missing or invalid data
Processed entry 2 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-00/output2.geojson
Processed entry 3 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-00/output3.geojson
Skipped entry 4 due to missing or invalid data
Skipped entry 5 due to missing or invalid data
Skipped entry 6 due to missing or invalid data


'URL: https://aurorasaurus.org/oval-data?end_date=2024-05-10T15:15&local_offset=0'

Skipped entry 1 due to missing or invalid data
Processed entry 2 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-15/output2.geojson
Processed entry 3 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-15/output3.geojson
Skipped entry 4 due to missing or invalid data
Skipped entry 5 due to missing or invalid data
Skipped entry 6 due to missing or invalid data


'URL: https://aurorasaurus.org/oval-data?end_date=2024-05-10T15:30&local_offset=0'

...

Processed entry 1 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-30/output1.geojson
Processed entry 2 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-30/output2.geojson
Skipped entry 3 due to missing or invalid data
Processed entry 4 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-30/output4.geojson
Skipped entry 5 due to missing or invalid data
Processed entry 6 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-30/output6.geojson
Skipped entry 7 due to missing or invalid data
Processed entry 8 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-30/output8.geojson
Skipped entry 9 due to missing or invalid data
Skipped entry 10 due to missing or invalid data
Skipped entry 11 due to missing or invalid data


'URL: https://aurorasaurus.org/oval-data?end_date=2024-05-11T23:45&local_offset=0'

Processed entry 1 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-45/output1.geojson
Skipped entry 2 due to missing or invalid data
Processed entry 3 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-45/output3.geojson
Processed entry 4 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-45/output4.geojson
Skipped entry 5 due to missing or invalid data
Processed entry 6 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-45/output6.geojson
Skipped entry 7 due to missing or invalid data
Processed entry 8 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-45/output8.geojson
Skipped entry 9 due to missing or invalid data
Processed entry 10 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-11-23-45/output10.geojson
Skipped entry 11 due to missing or invalid data
Skipped entry 12 due to missing or i

'URL: https://aurorasaurus.org/oval-data?end_date=2024-05-12T00:00&local_offset=0'

Skipped entry 1 due to missing or invalid data
Processed entry 2 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-12-00-00/output2.geojson
Processed entry 3 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-12-00-00/output3.geojson
Skipped entry 4 due to missing or invalid data
Processed entry 5 saved to C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-12-00-00/output5.geojson
Skipped entry 6 due to missing or invalid data
Skipped entry 7 due to missing or invalid data
Skipped entry 8 due to missing or invalid data
Skipped entry 9 due to missing or invalid data
Skipped entry 10 due to missing or invalid data
Skipped entry 11 due to missing or invalid data


OVAL ASSET GENERATOR

In [ ]:
import os
import json
from datetime import datetime

# Directory where GeoJSON files are stored
output_directory = 'C:/Users/alundkvi/Documents/work/scripts/oval_output_only_south/'

# Output directory for .asset files
asset_output_directory = 'C:/Users/alundkvi/Documents/work/scripts/oval_only_south_assets/'

# Function to generate .asset content for a folder of GeoJSON files
def generate_asset(folder_path):
    # Extract date and time from folder name
    folder_name = os.path.basename(folder_path)
    date_time_str = folder_name.replace('_', ' ')
    date_time = datetime.strptime(date_time_str, "%Y-%m-%d %H-%M-%S")

    # Define Lua script content template
    lua_script = f"""
local earth = asset.require("scene/solarsystem/planets/earth/earth")

local geojsonPath = asset.resource("C:/Users/alundkvi/Documents/work/OpenSpace/user/data/assets/aurorasaurus/geojson/oval_only_south_geojsons/{folder_name}/")

"""
# Calculate start and end times for the 15-minute interval
    start_time_str = date_time.strftime("%Y %B %d %H:%M:%S")
    end_time = date_time + timedelta(minutes=15)
    end_time_str = end_time.strftime("%Y %B %d %H:%M:%S")
    # Process each GeoJSON file in the folder
    file_index = 1
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.geojson'):
            file_path = os.path.join(folder_path, filename)
            # Generate Lua script for each GeoJSON file
            lua_script += f"""
local AuroraOval{file_index} = {{
  Identifier = "AuroraOval_{date_time_str}_{file_index}",
  File = geojsonPath .. "{filename}",
  TimeFrame = {{
    Type = "TimeFrameInterval",
    Start = "{start_time_str}",
    End = "{end_time_str}"
  }},
  HeightOffset = 75000,
  Name = "AuroraOval_{date_time_str}_{file_index}"
}}

asset.onInitialize(function()

  openspace.globebrowsing.addGeoJson(earth.Earth.Identifier, AuroraOval{file_index})

end)

asset.onDeinitialize(function()

  openspace.globebrowsing.deleteGeoJson(earth.Earth.Identifier, AuroraOval{file_index})

end)

"""
            file_index += 1

            # Move to the next 15-minute interval
            date_time = end_time

    # Write Lua script to .asset file in asset_output_directory
    asset_filename = f'{folder_name}.asset'
    asset_filepath = os.path.join(asset_output_directory, asset_filename)
    with open(asset_filepath, 'w') as asset_file:
        asset_file.write(lua_script)

    print(f'Generated {asset_filename} in {asset_output_directory}')

# Ensure asset output directory exists
os.makedirs(asset_output_directory, exist_ok=True)

# Iterate through each folder in output_directory
for folder_name in sorted(os.listdir(output_directory)):
    folder_path = os.path.join(output_directory, folder_name)
    if os.path.isdir(folder_path):
        generate_asset(folder_path)

In [ ]:
from datetime import datetime, timedelta

# Define start and end datetime
start_datetime = datetime(2024, 5, 10, 15, 0, 0)
end_datetime = datetime(2024, 5, 12, 12, 0, 0)

# Define interval duration (15 minutes)
interval = timedelta(minutes=15)

# Function to generate required asset paths
def generate_asset_requirements(start, end, interval):
    current = start
    while current <= end:
        asset_path = f'asset.require("./oval_only_south_assets/{current.strftime("%Y-%m-%d_%H-%M-%S")}")'
        print(asset_path)
        current += interval

# Generate asset requirements
generate_asset_requirements(start_datetime, end_datetime, interval)


In [ ]:
import shapely
print(shapely.__version__)


VIEWLINE GEOJSON GENERATOR

In [18]:
import requests
import json
from datetime import datetime, timedelta
import os

# Function to fetch data from URL and adjust coordinates
def fetch_northern_coordinates(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        northern_coordinates = data.get('northern', [])
        # Adjust lng coordinates by removing 360
        adjusted_coordinates = [(item['lng'] - 360, item['lat']) for item in northern_coordinates]
        return adjusted_coordinates
    else:
        print(f"Failed to fetch data from {url}")
        return []
    
    
# Function to fetch data from URL and adjust coordinates
def fetch_southern_coordinates(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        southern_coordinates = data.get('southern', [])
        # Adjust lng coordinates by removing 360
        adjusted_coordinates = [(item['lng'] - 360, item['lat']) for item in southern_coordinates]
        return adjusted_coordinates
    else:
        print(f"Failed to fetch data from {url}")
        return []

# Function to create LineString GeoJSON and save to file
def create_geojson_file(coordinates, output_path):
    if not os.path.exists(os.path.dirname(output_path)):
        os.makedirs(os.path.dirname(output_path))
        
    feature_collection = {
        "type": "FeatureCollection",
        "features": []
    }

    line_feature = {
        "type": "Feature",
        "properties": {
            "name": "Aurora viewline",
            "stroke": "#FF5733"
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        }
    }

    feature_collection["features"].append(line_feature)

    with open(output_path, 'w') as f:
        json.dump(feature_collection, f, indent=4)

# Define output directory
output_directory = "C:/Users/alundkvi/Documents/work/scripts/southern_viewline_asset_output/"

# Define start and end datetime
start_datetime = datetime(2024, 5, 10, 15, 0, 0)
end_datetime = datetime(2024, 5, 12, 0, 0, 0)
interval = timedelta(minutes=15)

# Iterate through each 15-minute interval
current_datetime = start_datetime
while current_datetime <= end_datetime:
    # Construct URL for the current interval
    formatted_datetime = current_datetime.strftime("%Y-%m-%dT%H:%M:%S%z")
    url = f"https://aurorasaurus.org/view-lines?end_date={formatted_datetime}&format=json&local_offset=0"

    # Fetch northern coordinates from URL
    #northern_coordinates = fetch_northern_coordinates(url)

    # Fetch southern coordinates from URL
    southern_coordinates = fetch_southern_coordinates(url)

    # Create output filename based on datetime
    output_filename = current_datetime.strftime("%Y-%m-%d_%H-%M-%S") + ".geojson"
    output_path = os.path.join(output_directory, output_filename)

    # Create GeoJSON file with northern coordinates in the specified format
    #create_geojson_file(northern_coordinates, output_path)
    
    # Create GeoJSON file with southern coordinates in the specified format
    create_geojson_file(southern_coordinates, output_path)

    # Move to the next interval
    current_datetime += interval


VIEWLINE ASSET GENERATOR


In [21]:
import os
import json
from datetime import datetime

# Directory where GeoJSON files are stored
output_directory = 'C:/Users/alundkvi/Documents/work/scripts/southern_viewline_asset_output/'

# Output directory for .asset files
asset_output_directory = 'C:/Users/alundkvi/Documents/work/scripts/southern_viewline_asset/'

# Function to generate .asset content for a folder of GeoJSON files
def generate_asset(folder_path):
    # Define Lua script content template
    lua_script = f"""
local earth = asset.require("scene/solarsystem/planets/earth/earth")

local geojsonPath = asset.resource("geojson/southern_viewline_geojsons/")

"""

    # Process each GeoJSON file in the folder
    file_index = 1
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.geojson'):
             # Extract date and time from file name
            date_time_str = filename.replace('_', ' ')
            date_time_str = date_time_str.replace('.geojson', '')
            date_time = datetime.strptime(date_time_str, "%Y-%m-%d %H-%M-%S")

        # Calculate start and end times for the 15-minute interval
            start_time_str = date_time.strftime("%Y %B %d %H:%M:%S")
            end_time = date_time + timedelta(minutes=15)
            end_time_str = end_time.strftime("%Y %B %d %H:%M:%S")
            file_path = os.path.join(folder_path, filename)
            # Generate Lua script for each GeoJSON file
            lua_script += f"""
local southern_viewline_{file_index} = {{
  Identifier = "southern_viewline_{date_time_str}_{file_index}",
  File = geojsonPath .. "{filename}",
  TimeFrame = {{
    Type = "TimeFrameInterval",
    Start = "{start_time_str}",
    End = "{end_time_str}"
  }},
  HeightOffset = 75000,
  Name = "southern_viewline_{date_time_str}_{file_index}"
}}

"""
            file_index += 1

            # Move to the next 15-minute interval
            date_time = end_time


    lua_script += f"""
asset.onInitialize(function()
"""

    for i in range(1, file_index):
      lua_script += f"""openspace.globebrowsing.addGeoJson(earth.Earth.Identifier, southern_viewline_{i})
"""
    lua_script += f"""
end)
"""


    lua_script += f"""
asset.onDeinitialize(function()
"""
    for i in range(1, file_index):
      lua_script += f"""openspace.globebrowsing.deleteGeoJson(earth.Earth.Identifier, southern_viewline_{i})
"""
    lua_script += f"""
end)

"""
    # Write Lua script to .asset file in asset_output_directory
    asset_filename = 'southernViewline.asset'
    asset_filepath = os.path.join(asset_output_directory, asset_filename)
    with open(asset_filepath, 'w') as asset_file:
        asset_file.write(lua_script)

    print(f'Generated {asset_filename} in {asset_output_directory}')

# Ensure asset output directory exists
os.makedirs(asset_output_directory, exist_ok=True)

generate_asset(output_directory)

Generated southernViewline.asset in C:/Users/alundkvi/Documents/work/scripts/southern_viewline_asset/


In [ ]:
!pip install geopandas

In [ ]:
import json
from PIL import Image, ImageDraw
from shapely.geometry import shape
from pyproj import Transformer

# Read GeoJSON file
geojson_file = 'C:/Users/alundkvi/Documents/work/OpenSpace/user/data/assets/aurorasaurus/geojson/oval_geojsons/2024-05-10_19-00-00/output2.geojson'

with open(geojson_file, 'r') as f:
    geojson_data = json.load(f)

# Check if the file is a FeatureCollection or a single Feature
if geojson_data['type'] == 'FeatureCollection':
    features = geojson_data['features']
elif geojson_data['type'] == 'Feature':
    features = [geojson_data]
else:
    raise ValueError('Unsupported GeoJSON format')

# Calculate image size and bounds
width, height = 5400, 2700  # Image dimensions
xmin, ymin, xmax, ymax = float('inf'), float('inf'), float('-inf'), float('-inf')

for feature in features:
    geom = shape(feature['geometry'])
    xmin = min(xmin, geom.bounds[0])
    ymin = min(ymin, geom.bounds[1])
    xmax = max(xmax, geom.bounds[2])
    ymax = max(ymax, geom.bounds[3])

print(f"xmin: {xmin}, ymin: {ymin}, xmax: {xmax}, ymax: {ymax}")

# Create a blank image canvas
image = Image.new('RGBA', (width, height), (255, 255, 255, 0))  # Transparent background
draw = ImageDraw.Draw(image)

# Define a function to scale and project coordinates to fit image dimensions
transformer = Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)

def project_and_scale(x, y):
    projected_x, projected_y = transformer.transform(x, y)
    scaled_x = width * (projected_x - xmin) / (xmax - xmin)
    scaled_y = height * (ymax - projected_y) / (ymax - ymin)
    return (scaled_x, height - scaled_y)  # Flip y-axis for image coordinates

# Draw each polygon with its specified fill color
for feature in features:
    geom = shape(feature['geometry'])
    coords = [project_and_scale(x, y) for x, y in geom.exterior.coords]
    fill_color = feature['properties']['fill']
    draw.polygon(coords, fill=fill_color, outline='black')

output_file = 'geojsonImages/output_image.png'
image.save(output_file)
print(f"Image saved: {output_file}")

# Show image (optional for verification)
image.show()


In [ ]:
!pip install matplotlib geopandas

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from shapely.geometry import shape
from PIL import Image

# Load GeoJSON file
geojson_file = 'C:/Users/alundkvi/Documents/work/OpenSpace/user/data/assets/aurorasaurus/geojson/oval_geojsons/2024-05-10_19-00-00/output2.geojson'
gdf = gpd.read_file(geojson_file)

# Initialize matplotlib figure
fig, ax = plt.subplots(figsize=(54, 27))  # Size in inches (5400 / 100, 2700 / 100)

# Iterate over each feature in GeoDataFrame
for feature in gdf.iterrows():
    geom = shape(feature[1]['geometry'])
    if geom.geom_type == 'Polygon':
        coords = geom.exterior.coords.xy
        poly = Polygon(list(zip(coords[0], coords[1])), closed=True)
        #fill_color = feature['properties']['fill']
        print(coords)
        ax.add_patch(poly)

# Set limits and aspect ratio
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)
ax.set_aspect('equal')

# Save the plot as an image
plt.axis('off')
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust margins
plt.savefig('geojsonImages/output_image.png', dpi=100, bbox_inches='tight', pad_inches=0)
plt.close()

# Resize image to 5400 x 2700 using PIL
image = Image.open('geojsonImages/output_image.png')
resized_image = image.resize((5400, 2700), Image.ANTIALIAS)
resized_image.save('geojsonImages/resized_output_image.png')

In [ ]:
!pip install GDAL


In [ ]:
!pip install Fiona


In [ ]:
!pip install Shapely

In [ ]:
!pip uninstall numpy

In [ ]:
!pip install Fiona

In [ ]:
!pip install numpy==1.21.2

In [55]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from PIL import Image

# Load GeoJSON file
geojson_file = 'C:/Users/alundkvi/Documents/work/scripts/oval_asset_output_not_simplified/2024-05-10_19-00-00/output1.geojson'
gdf = gpd.read_file(geojson_file)

# Initialize matplotlib figure with the desired aspect ratio
fig_neg, ax_neg = plt.subplots(figsize=(54, 27))  # Size in inches (5400 / 100, 2700 / 100) for negative longitudes
fig_pos, ax_pos = plt.subplots(figsize=(54, 27))  # Size in inches (5400 / 100, 2700 / 100) for positive longitudes

# Set global plot limits for entire Earth
ax_neg.set_xlim(-180, 180)
ax_neg.set_ylim(-90, 90)
ax_neg.set_aspect('equal')

ax_pos.set_xlim(-180, 180)
ax_pos.set_ylim(-90, 90)
ax_pos.set_aspect('equal')

# Function to correct longitude to be within -180 to 180 range
def correct_longitude(lon):
    while lon > 180:
        lon -= 360
    while lon < -180:
        lon += 360
    return lon


# Lists to hold coordinates for negative and positive longitudes
negative_coords = []
positive_coords = []

# Iterate over each feature in GeoDataFrame
for index, row in gdf.iterrows():
    geom = row['geometry']
    if geom.geom_type == 'Polygon':
        
        # Extract fill color - adjust according to your GeoJSON structure
        if 'fill' in row:  # Check if 'fill' property exists directly
            fill_color = row['fill']
        elif 'properties' in row and 'fill' in row['properties']:  # Check nested structure
            fill_color = row['properties']['fill']
        else:
            fill_color = '#a6ff00'  # Default fill color if not found
        
        coords = geom.exterior.coords.xy

        # Handle wrapping around the globe for longitude and ensure closure
        wrapped_coords = [(correct_longitude(x), y) for x, y in zip(coords[0], coords[1])]
        
        
        # Split wrapped coordinates into positive and negative based on longitude
        positive_polygon = []
        negative_polygon = []
        
        for lon, lat in wrapped_coords:
            if lon >= -90:
                positive_polygon.append((lon, lat))
            else:
                negative_polygon.append((lon, lat))
        
        # Add points to ensure polygons extend to image edges
        if positive_polygon:
            # Add point at 180 longitude if not already present
            if not any(lon == -180 for lon, lat in negative_polygon):
                print('runns')
                max_lat = max(lat for lon, lat in positive_polygon)
                min_lat = min(lat for lon, lat in positive_polygon)
                positive_polygon.append((180, max_lat))
                positive_polygon.append((180, min_lat))
            positive_coords.append(positive_polygon)
        
        if negative_polygon:
            # Add point at -180 longitude if not already present
            if not any(lon == -180 for lon, lat in negative_polygon):
                print('runns')
                min_lat2 = min(lat for lon, lat in positive_polygon)
                max_lat2 = max(lat for lon, lat in negative_polygon)
                negative_polygon.insert(0,(-180, max_lat2))
                negative_polygon.insert(0,(-180, min_lat2))
            negative_coords.append(negative_polygon)

# Create polygons for negative longitudes
for coords in negative_coords:
    poly_neg = Polygon(coords, closed=True)
    poly_neg.set_facecolor(fill_color) 
    ax_neg.add_patch(poly_neg)

# Create polygons for positive longitudes
for coords in positive_coords:
    poly_pos = Polygon(coords, closed=True)
    poly_pos.set_facecolor(fill_color) 
    ax_pos.add_patch(poly_pos)

# Save the plots as images with transparent backgrounds
ax_neg.axis('off')
fig_neg.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust margins for fig_neg
fig_neg.savefig('geojsonImages/negative_longitude_output.png', dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
plt.close(fig_neg)

ax_pos.axis('off')
fig_pos.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust margins for fig_pos
fig_pos.savefig('geojsonImages/positive_longitude_output.png', dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
plt.close(fig_pos)

# Resize images to 5400 x 2700 using PIL and make background transparent
image_neg = Image.open('geojsonImages/negative_longitude_output.png').convert('RGBA')
resized_image_neg = image_neg.resize((5400, 2700), Image.ANTIALIAS)
resized_image_neg.save('geojsonImages/resized_negative_longitude_output.png', format='PNG')

image_pos = Image.open('geojsonImages/positive_longitude_output.png').convert('RGBA')
resized_image_pos = image_pos.resize((5400, 2700), Image.ANTIALIAS)
resized_image_pos.save('geojsonImages/resized_positive_longitude_output.png', format='PNG')


runns
runns


C:\Users\alundkvi\AppData\Local\Temp\1\ipykernel_40232\2050354773.py:111: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_image_neg = image_neg.resize((5400, 2700), Image.ANTIALIAS)


KeyboardInterrupt: 

In [7]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import geopandas as gpd
import numpy as np
import os

# Function to handle longitude wrapping and splitting into separate polygons
def wrap_and_split_polygon(coords):
    split_coords = []
    remaining_coords = []
    for i in range(len(coords)):
        lon, lat = coords[i]
        if lon > 180:
            split_coords.append((lon - 360, lat))
        elif lon < -180:
            split_coords.append((lon + 360, lat))
        else:
            remaining_coords.append((lon, lat))
    
    return split_coords, remaining_coords

# Function to add points to extend polygons to image edges
def extend_to_edge(coords):
    if len(coords) > 1:
        lon1, lat1 = coords[0]
        lon2, lat2 = coords[-1]
        if lon1 != lon2:  # Only add points if there's a wrap-around
            coords.append((180 if lon1 < lon2 else -180, lat1))
            coords.append((180 if lon1 < lon2 else -180, lat2))

def find_closest_to_minus180(coords):
    closest_idx = None
    closest_diff = float('inf')

    for idx, (lon, lat) in enumerate(coords):
        if abs(lon + 180) < closest_diff:
            closest_diff = abs(lon + 180)
            closest_idx = idx

    return closest_idx

def find_closest_to_180(coords):
    closest_idx = None
    closest_diff = float('inf')

    for idx, (lon, lat) in enumerate(coords):
        if abs(lon - 180) < closest_diff:
            closest_diff = abs(lon - 180)
            closest_idx = idx

    return closest_idx

# Root directory containing folders with GeoJSON files
root_dir = 'C:/Users/alundkvi/Documents/work/scripts/oval_asset_output_not_simplified/temp/temp2/'

# Iterate over all subdirectories and their files
for dirpath, _, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.geojson'):
            # Read GeoJSON file
            geojson_file = os.path.join(dirpath, filename)

            gdf = gpd.read_file(geojson_file)

            # Extract polygon coordinates
            polygon_coords = np.array(gdf['geometry'][0].exterior.coords.xy).T

            # Extract fill color from GeoJSON properties
            fill_color = gdf['fill'][0]

            # Apply wrapping and splitting to polygon coordinates
            split_coords, coords = wrap_and_split_polygon(polygon_coords)
            #print(split_coords)
            #print("Next")
            #print(coords)

            # Create a figure and axis with specific dimensions
            fig, ax = plt.subplots(figsize=(54, 27))

            # Plot each split polygon separately
            if split_coords:
                split_coords_lon_average = sum(split_coords[0]) / len(split_coords[0])

                if split_coords_lon_average > 0:

                    closest_index2 = find_closest_to_180(split_coords)

                    if closest_index2 == 0:
                        lon1, lat1 = split_coords[closest_index2]
                        split_coords.insert(closest_index2, (190, lat1))
                        lon2, lat2 = split_coords[len(split_coords)-1]
                        split_coords.append((190, lat2))
                    elif closest_index2 == len(split_coords)-1:
                        lon1, lat1 = split_coords[len(split_coords)-1]
                        lon2, lat2 = split_coords[0]
                        split_coords.insert(0, (190, lat2))
                        split_coords.append((190, lat1))
                    else:
                        if closest_index2 is not None:
                                lon1, lat1 = split_coords[closest_index2]
                                #coords.insert(closest_index, (190, lat1))
                                #insert_values.append((closest_index,(190,lat1)))

                                next_index_lon, next_index_lat = split_coords[closest_index2 + 1] if closest_index2 + 1 < len(split_coords) else None
                                prev_index_lon, prev_index_lat = split_coords[closest_index2 - 1] if closest_index2 - 1 >= 0 else None

                                #print("Denna testar du + ")
                                if abs(next_index_lat - lat1) > abs(prev_index_lat - lat1):
                                    split_coords.insert(closest_index2, (190, next_index_lat))
                                    split_coords.insert(closest_index2, (190, lat1))
                                else:
                                    split_coords.insert(closest_index2, (190, lat1))
                                    split_coords.insert(closest_index2, (190, prev_index_lat))
                else:

                    closest_index2 = find_closest_to_minus180(split_coords)

                    if closest_index2 == 0:
                        lon1, lat1 = split_coords[closest_index2]
                        split_coords.insert(closest_index2, (-190, lat1))
                        lon2, lat2 = split_coords[len(split_coords)-1]
                        split_coords.append((-190, lat2))
                    elif closest_index2 == len(split_coords)-1:
                        lon1, lat1 = split_coords[len(split_coords)-1]
                        lon2, lat2 = split_coords[0]
                        split_coords.insert(0, (-190, lat2))
                        split_coords.append((-190, lat1))
                    else:
                        if closest_index2 is not None:
                                lon1, lat1 = split_coords[closest_index2]
                                #coords.insert(closest_index, (-190, lat1))

                                # Find the coordinate with the largest longitude just before or after the closest index
                                next_index_lon, next_index_lat = split_coords[closest_index2 + 1] if closest_index2 + 1 < len(split_coords) else None
                                prev_index_lon, prev_index_lat = split_coords[closest_index2 - 1] if closest_index2 - 1 >= 0 else None
                                #print("Denna testar du -")

                                if abs(next_index_lat - lat1) > abs(prev_index_lat - lat1):
                                    split_coords.insert(closest_index2, (-190, next_index_lat))
                                    split_coords.insert(closest_index2, (-190, lat1))
                                else:
                                    split_coords.insert(closest_index2, (-190, lat1))
                                    split_coords.insert(closest_index2, (-190, prev_index_lat))

                # Create a polygon patch
                polygon_patch = Polygon(split_coords, closed=True, edgecolor=fill_color, facecolor=fill_color, alpha=0.75)

                # Add polygon patch to axis
                ax.add_patch(polygon_patch)

            # Plot the remaining part of the polygon (if any)
            if coords:
                if split_coords:

                    coords_lon_average = sum(coords[0]) / len(coords[0])

                    # Determine insertion values based on the average longitude
                    insert_values = []

                    if coords_lon_average < 0:
                        # Find the index of the point where longitude is closest to -180
                        closest_index = find_closest_to_minus180(coords)

                        if closest_index == 0:
                            lon1, lat1 = coords[closest_index]
                            coords.insert(closest_index, (-190, lat1))
                            lon2, lat2 = coords[len(coords)-1]
                            coords.append((-190, lat2))
                        elif closest_index == len(coords)-1:
                            lon1, lat1 = coords[len(coords)-1]
                            lon2, lat2 = coords[0]
                            coords.insert(0, (-190, lat2))
                            coords.append((-190, lat1))
                        else:
                            if closest_index is not None:
                                lon1, lat1 = coords[closest_index]
                                #coords.insert(closest_index, (-190, lat1))

                                # Find the coordinate with the largest longitude just before or after the closest index
                                next_index_lon, next_index_lat = coords[closest_index + 1] if closest_index + 1 < len(coords) else None
                                prev_index_lon, prev_index_lat = coords[closest_index - 1] if closest_index - 1 >= 0 else None
                                #print("Denna testar du -")
                                
                                if abs(next_index_lat - lat1) > abs(prev_index_lat - lat1):
                                    coords.insert(closest_index, (-190, next_index_lat))
                                    coords.insert(closest_index, (-190, lat1))
                                else:
                                    coords.insert(closest_index, (-190, lat1))
                                    coords.insert(closest_index, (-190, prev_index_lat))

                                """ if closest_index - 1 >= 0 and coords[closest_index - 1] is not None:
                                    lon2, lat2 = coords[closest_index - 1]
                                    insert_values.append((closest_index, (-190, lat2)))

                                if closest_index - 2 >= 0 and coords[closest_index - 2] is not None:
                                    lon3, lat3 = coords[closest_index - 2]
                                    insert_values.append((closest_index - 1, (-190, lat3)))

                                if closest_index + 1 < len(coords) and coords[closest_index + 1] is not None:
                                    lon4, lat4 = coords[closest_index + 1]
                                    insert_values.append((closest_index + 2, (-190, lat4)))

                                if closest_index + 2 < len(coords) and coords[closest_index + 2] is not None:
                                    lon5, lat5 = coords[closest_index + 2]
                                    insert_values.append((closest_index + 3, (-190, lat5))) """

                    elif coords_lon_average > 0:
                        # Find the index of the point where longitude is closest to 180
                        closest_index = find_closest_to_180(coords)
                        if closest_index == 0:
                            lon1, lat1 = coords[closest_index]
                            coords.insert(closest_index, (190, lat1))
                            lon2, lat2 = coords[len(coords)-1]
                            coords.append((190, lat2))
                        elif closest_index == len(coords)-1:
                            lon1, lat1 = coords[len(coords)-1]
                            lon2, lat2 = coords[0]
                            coords.insert(0, (190, lat2))
                            coords.append((190, lat1))
                        else:
                            if closest_index is not None:
                                lon1, lat1 = coords[closest_index]
                                #coords.insert(closest_index, (190, lat1))
                                #insert_values.append((closest_index,(190,lat1)))

                                next_index_lon, next_index_lat = coords[closest_index + 1] if closest_index + 1 < len(coords) else None
                                prev_index_lon, prev_index_lat = coords[closest_index - 1] if closest_index - 1 >= 0 else None

                                #print("Denna testar du + ")
                                if abs(next_index_lat - lat1) > abs(prev_index_lat - lat1):
                                    coords.insert(closest_index + 1, (190, next_index_lat))
                                    coords.insert(closest_index + 1, (190, lat1))
                                else:
                                    coords.insert(closest_index, (190, lat1))
                                    coords.insert(closest_index, (190, prev_index_lat))


                                """ if closest_index + 1 < len(coords) and coords[closest_index + 1] is not None:
                                    lon2, lat2 = coords[closest_index + 1]
                                    insert_values.append((closest_index + 2, (190, lat2)))

                                if closest_index - 1 >= 0 and coords[closest_index - 1] is not None:
                                    lon3, lat3 = coords[closest_index - 1]
                                    insert_values.append((closest_index, (190, lat3)))

                                if closest_index - 2 >= 0 and coords[closest_index - 2] is not None:
                                    lon4, lat4 = coords[closest_index - 2]
                                    insert_values.append((closest_index - 1, (190, lat4)))

                                if closest_index + 2 < len(coords) and coords[closest_index + 2] is not None:
                                    lon5, lat5 = coords[closest_index + 2]
                                    insert_values.append((closest_index + 3, (190, lat5))) """

                    """ # Apply the inserts in reverse order to maintain correct positions
                    insert_values.sort(reverse=True, key=lambda x: x[0])
                    for index, coord in insert_values:
                        coords.insert(index, coord) """
                        
                    print(coords)

                # Create a polygon patch for the remaining part
                polygon_patch = Polygon(coords, closed=True, edgecolor=fill_color, facecolor=fill_color, alpha=0.75)

                # Add polygon patch to axis
                ax.add_patch(polygon_patch)

            # Set axis limits and aspect ratio
            ax.set_xlim(-180, 180)
            ax.set_ylim(-90, 90)
            ax.set_aspect('equal')

            # Remove axes
            ax.axis('off')

            # Output image file path
            output_file = os.path.splitext(filename)[0] + '.png'
            output_path = os.path.join(dirpath, output_file)

            # Save the image
            plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
            plt.close()  # Close the current figure to free up memory

[(190, 60.207053), (179.382043, 60.207053), (178.307105, 60.226625), (177.231844, 60.242213), (175.887334, 60.256224), (175.080407, 60.261773), (174.273323, 60.265223), (172.927847, 60.266402), (171.312759, 60.260476), (169.966452, 60.249614), (168.350447, 60.229713), (167.003444, 60.207599), (165.386683, 60.174671), (164.039118, 60.142097), (162.421764, 60.09709), (160.804154, 60.045895), (159.186336, 59.988793), (157.568358, 59.926066), (155.950269, 59.857998), (154.332116, 59.784869), (152.444254, 59.693533), (151.09581, 59.62456), (149.477754, 59.537944), (148.12947, 59.462749), (146.511686, 59.369141), (144.894118, 59.272119), (143.276815, 59.171965), (141.659824, 59.068961), (139.773794, 58.945565), (136.811207, 58.745672), (133.312178, 58.502114), (130.084829, 58.272601), (122.029628, 57.69421), (119.617401, 57.523306), (117.475114, 57.373796), (115.334749, 57.227246), (113.196419, 57.084324), (111.060239, 56.9457), (109.192933, 56.828454), (106.79478, 56.684017), (104.66573, 56

LAYER OVAL THIS IS THE ACTUAL ONE

In [4]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import geopandas as gpd
import numpy as np
import os
from PIL import Image

# Root directory containing folders with GeoJSON files
root_dir = 'C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/temp/'

# Function to determine the shift needed for longitudes to fit within -180 to 180
def calculate_shift_amounts(coords):
    longitudes = coords[:, 0]
    shift = 0
    plus = True
    abs_max = abs(np.max(longitudes))
    abs_min = abs(np.min(longitudes))

    if abs_max > abs_min and np.max(longitudes) > 180:
        shift = np.max(longitudes) - 180
        plus = False
    elif abs_max < abs_min and np.min(longitudes) < -180:
        shift = -180 - np.min(coords[:, 0])
        plus = True

    return shift, plus

def shift_image(image_path, degrees, output_path):
    # Load the image
    image = Image.open(image_path).convert("RGBA")  # Ensure the image is in RGBA mode to handle transparency
    width, height = image.size
    
    # Calculate how many pixels to shift
    pixels_per_degree = width / 360
    shift_pixels = int(degrees * pixels_per_degree)

    # Create a new image with transparent background
    new_image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

    # Split the image into two parts, considering transparency
    right_part = image.crop((width - shift_pixels, 0, width, height))
    left_part = image.crop((0, 0, width - shift_pixels, height))

    # Paste the shifted parts onto the new image
    new_image.paste(right_part, (0, 0), right_part)  # Use the image itself as a mask to preserve transparency
    new_image.paste(left_part, (shift_pixels, 0), left_part)  # Use the image itself as a mask to preserve transparency

    # Save the result
    new_image.save(output_path)
    
def shift_image_left(image_path, degrees, output_path):
    # Load the image
    image = Image.open(image_path).convert("RGBA")  # Ensure the image is in RGBA mode to handle transparency
    width, height = image.size
    
    # Calculate how many pixels to shift
    pixels_per_degree = width / 360
    shift_pixels = int(degrees * pixels_per_degree)

    # Create a new image with transparent background
    new_image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

    # Split the image into two parts, considering transparency
    left_part = image.crop((0, 0, shift_pixels, height))
    right_part = image.crop((shift_pixels, 0, width, height))

    # Paste the shifted parts onto the new image
    new_image.paste(left_part, (width - shift_pixels, 0), left_part)  # Paste left part at the end to wrap around
    new_image.paste(right_part, (0, 0), right_part)  # Paste right part in its original position

    # Save the result
    new_image.save(output_path)

# Function to adjust coordinates based on shift amounts
def adjust_longitudes(coords, shift, plus):
    if plus == False:
        coords[:, 0] -= shift
    else:
        coords[:, 0] += shift
    return coords

# Iterate over all subdirectories and their files
for dirpath, _, filenames in os.walk(root_dir):
    geojson_files = [f for f in filenames if f.endswith('.geojson')]

    if not geojson_files:
        continue

    # Process each GeoJSON file to determine the maximum and minimum longitudes
    all_coords = []
    for filename in geojson_files:
        geojson_file = os.path.join(dirpath, filename)
        gdf = gpd.read_file(geojson_file)
        polygon_coords = np.array(gdf['geometry'][0].exterior.coords.xy).T
        all_coords.append(polygon_coords)

    # Flatten list of coordinates
    all_coords = np.vstack(all_coords)

    # Calculate the shift amounts
    shift, plus = calculate_shift_amounts(all_coords)

    print(dirpath)
    print(shift)
    #print(" ")

    # Apply the shift to each file
    for filename in geojson_files:
        geojson_file = os.path.join(dirpath, filename)
        gdf = gpd.read_file(geojson_file)
        polygon_coords = np.array(gdf['geometry'][0].exterior.coords.xy).T

        # Adjust longitudes
        adjusted_coords = adjust_longitudes(polygon_coords, shift, plus)

        # Extract fill color from GeoJSON properties
        fill_color = gdf['fill'][0]

        # Create a figure and axis with specific dimensions
        fig, ax = plt.subplots(figsize=(72, 36))

        # Create a polygon patch for the adjusted coordinates
        if len(adjusted_coords) > 0:
            polygon_patch = Polygon(adjusted_coords, closed=True, edgecolor=fill_color, facecolor=fill_color, alpha=1)
            ax.add_patch(polygon_patch)

        # Set axis limits and aspect ratio
        ax.set_xlim(-180, 180)
        ax.set_ylim(-90, 90)
        ax.set_aspect('equal')

        # Remove axes
        ax.axis('off')

        # Output image file path
        output_file = os.path.splitext(filename)[0] + '.png'
        output_path = os.path.join(dirpath, output_file)

        # Save the image
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close()  # Close the current figure to free up memory

        shift_image_left(output_path, shift, output_path)


C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-00
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-15
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-30
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-15-45
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-16-00
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-16-15
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-16-30
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-16-45
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-17-00
6.916502000000008
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-17-15
15.046841999999998
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-17-30
0
C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/2024-05-10-17-45
0
C:/User

In [1]:
from PIL import Image
import numpy as np
import os  # Make sure to import the os module

def add_transparency(image_path, output_path, left_lon, right_lon, top_lat, bottom_lat, image_width, image_height):
    # Load the original image
    image = Image.open(image_path).convert("RGBA")
    
    # Define the dimensions of the output image
    new_width = 3600  # Width in pixels for a 360° image
    new_height = 1800  # Height in pixels for a 180° image
    new_image = Image.new("RGBA", (new_width, new_height), (0, 0, 0, 0))
    
    # Calculate scaling factors
    lon_range = right_lon - left_lon
    lat_range = top_lat - bottom_lat
    
    scale_x = new_width / lon_range
    scale_y = new_height / lat_range
    
    # Calculate position to paste the original image
    left = int((left_lon - (-180)) * scale_x)
    top = int((90 - top_lat) * scale_y)
    
    # Paste the original image onto the transparent background
    new_image.paste(image, (left, top))
    
    # Save the new image
    new_image.save(output_path)


# Coordinates for the US
left_lon = -130  # Westernmost longitude of the image
right_lon = -60   # Easternmost longitude of the image
bottom_lat = 25.837592  # Southernmost latitude of Florida (approx)
top_lat = 55.837592     # Northernmost latitude of the US (approx)


# Example usage
add_transparency(
    image_path="C:/Users/alundkvi/Documents/work/scripts/conus-aurora-nolabels.png",
    output_path="C:/Users/alundkvi/Documents/work/scripts/us_image_transparent4.png",
    left_lon=left_lon,
    right_lon=right_lon,
    top_lat=top_lat,
    bottom_lat=bottom_lat,
    image_width=2212,
    image_height=1239
)


In [17]:
from PIL import Image
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

def add_transparency(image_path, output_path, left_lon, right_lon, top_lat, bottom_lat, image_width, image_height, scale_factor):
 # Load and resize the original image
    image = Image.open(image_path).convert("RGBA")
    scaled_width = int(image_width * scale_factor)
    scaled_height = int(image_height * scale_factor)
    image_resized = image.resize((scaled_width, scaled_height), Image.ANTIALIAS)
    
    # Define the size of the new image with transparency
    new_width = 3600  # Width for the equirectangular projection
    new_height = 1800  # Height for the equirectangular projection

    # Create a new image with a transparent background
    new_image = Image.new("RGBA", (new_width, new_height), (0, 0, 0, 0))

    # Setup the projection
    proj = ccrs.PlateCarree()
    
    # Setup figure and axis with Cartopy
    fig = plt.figure(figsize=(new_width / 100, new_height / 100), dpi=100)
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    
    # Set extent to match the coordinates
    ax.set_extent([left_lon, right_lon, bottom_lat, top_lat], crs=proj)
    
    # Add the image to the axis with transparency
    ax.imshow(np.array(image_resized), origin='upper', transform=proj, extent=[left_lon, right_lon, bottom_lat, top_lat], alpha=1.0)
    
    # Save the new image with transparency
    plt.savefig(output_path, format='png', bbox_inches='tight', pad_inches=0, dpi=100, transparent=True)
    plt.close(fig)
    
    print(f"Image saved successfully at {output_path}")


# Coordinates for the US
left_lon = -135  # Westernmost longitude of the image
right_lon = -55  # Easternmost longitude of the image
bottom_lat = 25.837592  # Southernmost latitude of Florida (approx)
top_lat = 60.078868     # Northernmost latitude of the US (approx)

# Example usage
add_transparency(
    image_path="C:/Users/alundkvi/Documents/work/scripts/conus-aurora-nolabels.png",
    output_path="C:/Users/alundkvi/Documents/work/scripts/us_image_transparent2.png",
    left_lon=left_lon,
    right_lon=right_lon,
    top_lat=top_lat,
    bottom_lat=bottom_lat,
    image_width=5544,
    image_height=2772,
    scale_factor=1.0  # Scale up by 5%
)

C:\Users\alundkvi\AppData\Local\Temp\4\ipykernel_5712\3876751299.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image_resized = image.resize((scaled_width, scaled_height), Image.ANTIALIAS)


Image saved successfully at C:/Users/alundkvi/Documents/work/scripts/us_image_transparent2.png
